# Dataverwerking michelson interferrometer

## Importeren packages

In [1]:
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.optimize as opt
import scipy.stats as nst

sp.init_printing()

In [2]:
#%matplotlib widget

In [3]:
mpl.rcParams['figure.dpi'] = 200

In [4]:
def error(func, variables, S_variables):
    return sp.sqrt(sum([sp.diff(func, var)**2 * S_var**2 for (var, S_var) in zip(variables, S_variables)]))

## Bepalen golflengte laser

Invoeren data

In [5]:
# array met het aantal getelde lijnen en de fout hierop
tellingen = np.array([30, 30, 50, 100, 50])
d_tellingen = np.ones_like(tellingen)/np.sqrt(3)
# array met hoeveel µm de spiegel verschoven was en de fout hierop
afstand = np.array([9.5, 9.5, 16, 31.5, 16])
d_afstand = np.ones_like(afstand)*0.5/np.sqrt(3)

formules voor de golflengte en de fout

In [6]:
x, N, d_x, d_N = sp.symbols("x, N, d_x, d_N")
lam = 2*x/N
d_lam = error(lam, [x, N], [d_x, d_N])

bereken de golflengtes en de fout erop voor de verschillende metingen

In [7]:
golflengtes = sp.lambdify([x, N, d_x, d_N], lam, "numpy")(afstand, tellingen,
                                                         d_afstand, d_tellingen)
d_golflengtes = sp.lambdify([x, N, d_x, d_N], d_lam, "numpy")(afstand, tellingen,
                                                         d_afstand, d_tellingen)
# neem een gewogen gemiddelde van de berekende golflengtes
g = 1/d_golflengtes**2
golflengte = np.dot(golflengtes, g)/np.sum(g)
d_golflengte = 1/np.sqrt(np.sum(g))
display(golflengte*1000, d_golflengte*1000)

## Bepalen brekingsindex

Importeren data

In [8]:
lucht_frame = pd.read_excel("druk-en-tellingen.xlsx", sheet_name="Lucht")
CO2_frame = pd.read_excel("druk-en-tellingen.xlsx", sheet_name="CO2")

In [9]:
p, dp = sp.symbols('p, d_p')
frac = N/(p*1000)
d_frac = error(frac, [N, p], [d_N, dp])

brekingsindex lucht

In [10]:
subs = {N : lucht_frame["tel"].values[-1]-1, p : lucht_frame["druk"].values[-1] - 
        lucht_frame["druk"].values[1], d_N : 1, dp : 1}
frac_lucht = frac.subs(subs)
dfrac_lucht = d_frac.subs(subs)

In [11]:
dNdP, L, lam, T, T0, p0, n0 = sp.symbols('dN/dP, L, lambda, T, T_0, p_0, n_0')
SdNdP, SL, Slam, ST, ST0, Sp0 = sp.symbols('S_dN/dP, S_L, S_lambda, S_T, S_T_0, S_p_0')
eq = sp.Eq(dNdP, 2*L/lam * (n0-1)/T * T0/p0)
n0 = sp.solve(eq, n0)[0]
S_n0 = error(n0, [dNdP, L, lam, T, T0, p0], [SdNdP, SL, Slam, ST, ST0, Sp0])
subs = {dNdP:frac_lucht, L:0.0384, lam:golflengte*1e-6, T:293.15, T0:273.15, p0:1.013e5,
       SdNdP:dfrac_lucht, SL:0, Slam:d_golflengte*1e-6, ST:1, ST0:0, Sp0:0}
display(n0.subs(subs), S_n0.subs(subs))

brekingsindex CO2

In [12]:
subs = {N : CO2_frame["tel"].values[-1]-1, p : CO2_frame["druk"].values[-1] - 
        CO2_frame["druk"].values[1], d_N : 1, dp : 1}
frac_lucht = frac.subs(subs)
dfrac_lucht = d_frac.subs(subs)

In [13]:
dNdP, L, lam, T, T0, p0, n0 = sp.symbols('dN/dP, L, lambda, T, T_0, p_0, n_0')
SdNdP, SL, Slam, ST, ST0, Sp0 = sp.symbols('S_dN/dP, S_L, S_lambda, S_T, S_T_0, S_p_0')
eq = sp.Eq(dNdP, 2*L/lam * (n0-1)/T * T0/p0)
n0 = sp.solve(eq, n0)[0]
S_n0 = error(n0, [dNdP, L, lam, T, T0, p0], [SdNdP, SL, Slam, ST, ST0, Sp0])
subs = {dNdP:frac_lucht, L:0.0384, lam:golflengte*1e-6, T:293.15, T0:273.15, p0:1.013e5,
       SdNdP:dfrac_lucht, SL:0, Slam:d_golflengte*1e-6, ST:1, ST0:0, Sp0:0}
display(n0.subs(subs), S_n0.subs(subs))